In [13]:
import pandas as pd
import numpy as np
import wrds
from pandas.tseries.offsets import *
import pickle as pkl
import pyarrow.feather as feather
import os
import warnings
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import ElasticNet
from sklearn.decomposition import PCA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.linear_model import LinearRegression, Lasso
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy.stats import spearmanr
import xgboost as xgb
from xgboost import XGBRegressor
from matplotlib.ticker import MaxNLocator
from typing import Dict, List
from sklearn.model_selection import TimeSeriesSplit
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader

warnings.filterwarnings("ignore", category=FutureWarning)

def v(x,y=5):
    print(x.shape)
    print(x.head(y))

def cm(df):
    cols_with_nan = df.columns[df.isna().any()].tolist()
    rows_with_nan = df[df.isna().any(axis=1)]
    print(cols_with_nan,rows_with_nan)

In [14]:
rank_imputed = pd.read_csv('D:/股票项目/chars60/chars60_rank_imputed.csv')
rank_imputed[:1000].to_excel('D:/股票项目/chars60/chars60_rank_impute.xlsx')
rank_imputed = rank_imputed[[col for col in rank_imputed.columns if col!='Unnamed: 0']]

In [15]:
data = rank_imputed.copy()
def fmm(data, group_col='permno'):
    # 1. 分离分组列与数值列
    if group_col not in data.columns:
        raise ValueError(f"分组列'{group_col}'不存在于数据中！")
    
    numeric_cols = data.select_dtypes(include=np.number).columns.drop(group_col)  # 提取数值列（排除分组列）
    if not numeric_cols.size:
        raise ValueError("数据中没有可填充的数值列！")
    
    # 2. 按分组列计算数值列的中位数（处理空分组和全NaN分组）
    group_medians = data.groupby(group_col)[numeric_cols].transform(
        lambda x: x.median() if x.notna().any() else np.nan  # 若分组全为NaN，暂存为NaN
    )
    
    # 3. 用全局中位数填充分组中位数中的NaN（避免全NaN分组无法填充）
    for col in numeric_cols:
        global_median = data[col].median()  # 计算全局中位数
        group_medians[col] = group_medians[col].fillna(global_median)  # 填充NaN分组
    
    # 4. 填充原始数据的缺失值
    data_filled = data.copy()
    data_filled[numeric_cols] = data_filled[numeric_cols].fillna(group_medians)
    
    # 5. 检查填充结果（可选但重要）
    missing_count = data_filled[numeric_cols].isna().sum().sum()
    if missing_count > 0:
        print(f"警告：填充后仍有 {missing_count} 个缺失值，可能因全NaN分组无法填充，建议手动处理或删除相关行。")
    else:
        print("缺失值填充成功！")
    return data_filled
data = fmm(data)
data = data.dropna()
v(data)

## 预处理
def preprocess_data(data):
    # 将日期转换为datetime格式并提取年、月
    data['date'] = pd.to_datetime(data['date'])
    data['year'] = data['date'].dt.year
    data['month'] = data['date'].dt.month
    
    # 移除包含缺失值的行
    data = data.dropna()
    
    # 创建滞后收益率特征
    data = data.sort_values(['permno', 'date'])
    data['lag_ret_1'] = data.groupby('permno')['ret'].shift(1)
    data['lag_ret_3'] = data.groupby('permno')['ret'].shift(3)
    # data['lag_ret_6'] = data.groupby('permno')['ret'].shift(6)
    
    # 移除包含NaN的行（由于滞后操作产生）
    data = data.dropna()
    
    return data

## 行业数据
def engineer_features(data):
    # 计算每个月的股票数量
    monthly_stocks = data.groupby(['year', 'month'])['permno'].count().reset_index()
    monthly_stocks.rename(columns={'permno': 'monthly_stocks'}, inplace=True)
    data = pd.merge(data, monthly_stocks, on=['year', 'month'], how='left')
    
    # 计算行业平均收益率
    industry_ret = data.groupby(['ffi49', 'year', 'month'])['ret'].mean().reset_index()
    industry_ret.rename(columns={'ret': 'industry_ret'}, inplace=True)
    data = pd.merge(data, industry_ret, on=['ffi49', 'year', 'month'], how='left')
    
    # 计算市场平均收益率
    market_ret = data.groupby(['year', 'month'])['ret'].mean().reset_index()
    market_ret.rename(columns={'ret': 'market_ret'}, inplace=True)
    data = pd.merge(data, market_ret, on=['year', 'month'], how='left')
    
    return data

processed_data = preprocess_data(data)
edata = engineer_features(processed_data)
edata

缺失值填充成功！
(332271, 70)
   gvkey  permno   sic       ret  exchcd  shrcd        date  ffi49  \
0   1004   54594  5080  0.029534     1.0   11.0  2016-10-31     42   
1   1004   54594  5080  0.146721     1.0   11.0  2016-11-30     42   
2   1004   54594  5080 -0.104093     1.0   11.0  2016-12-31     42   
3   1004   54594  5080 -0.029803     1.0   11.0  2017-01-31     42   
4   1004   54594  5080  0.075961     1.0   11.0  2017-02-28     42   

       lag_me  rank_mom36m  ...  rank_lev  rank_me_ia  rank_cfp  rank_hire  \
0  1070329.68     0.014351  ...  0.167702    0.621118  0.540373        1.0   
1  1099377.58     0.000000  ...  0.099602    0.657371  0.541833        0.0   
2  1260678.86     0.000000  ...  0.072897    0.648598  0.461682        0.0   
3  1134573.45     0.014351  ...  0.143363    0.610619  0.582301       -1.0   
4  1098184.71     0.000000  ...  0.123139    0.610284  0.610284        0.0   

   rank_cashdebt   rank_me  rank_ni  rank_roa  rank_grltnoa     log_me  
0            0.

,gvkey,permno,sic,ret,exchcd,shrcd,date,ffi49,lag_me,rank_mom36m,...,rank_roa,rank_grltnoa,log_me,year,month,lag_ret_1,lag_ret_3,monthly_stocks,industry_ret,market_ret
0,12825,10026,2050,-0.033289,3.0,11.0,2017-05-31,2,2.519068e+06,0.122405,...,0.667826,0.000000,14.739400,2017,5,-0.007229,0.048836,726,-0.045933,-0.008713
1,12825,10026,2050,0.018371,3.0,11.0,2017-06-30,2,2.435212e+06,0.000000,...,0.675408,0.480519,14.705544,2017,6,-0.033289,0.016293,764,-0.004490,0.015313
2,12825,10026,2050,-0.005073,3.0,11.0,2017-07-31,2,2.473539e+06,0.000000,...,0.671441,0.662420,14.721161,2017,7,0.018371,-0.007229,807,0.008039,0.000371
3,12825,10026,2050,-0.029756,3.0,11.0,2017-08-31,2,2.462304e+06,0.000000,...,0.839416,0.273617,14.716608,2017,8,-0.005073,-0.033289,3421,-0.027544,-0.016907
4,12825,10026,2050,0.033179,3.0,11.0,2017-09-30,2,2.389035e+06,0.000000,...,0.841316,0.271078,14.686400,2017,9,-0.029756,0.018371,3402,0.045498,0.073296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316382,184996,93436,3711,-0.077391,3.0,11.0,2024-08-31,23,7.413801e+08,0.110106,...,0.551696,-0.200882,20.424024,2024,8,0.172781,-0.028372,3679,-0.081720,-0.024281
316383,184996,93436,3711,0.221942,3.0,11.0,2024-09-30,23,6.840044e+08,0.141393,...,0.551432,-0.200127,20.343475,2024,9,-0.077391,0.111186,3658,-0.044205,0.003945
316384,184996,93436,3711,-0.045025,3.0,11.0,2024-10-31,23,8.390474e+08,-0.015357,...,0.547253,-0.198337,20.547778,2024,10,0.221942,0.172781,3629,-0.058508,-0.001507
316385,184996,93436,3711,0.381469,3.0,11.0,2024-11-30,23,8.020335e+08,-0.099911,...,0.651624,-0.232676,20.502661,2024,11,-0.045025,-0.077391,3624,0.046959,0.100034


### 遗忘模型

In [16]:
# 特征分类映射
feature_categories = {
    "动量特征": ['rank_mom1m', 'rank_mom6m', 'rank_mom12m', 'rank_mom36m', 'rank_mom60m'],
    "波动性特征": ['rank_rvar_mean', 'rank_std_dolvol', 'rank_rvar_ff3', 'rank_baspread', 
              'rank_std_turn', 'rank_rvar_capm', 'rank_maxret'],
    "基本面特征": ['rank_op', 'rank_rd_sale', 'rank_noa', 'rank_bm', 'rank_bm_ia', 'rank_acc',
             'rank_ato', 'rank_pm', 'rank_agr', 'rank_roe', 'rank_gma', 'rank_cfp',
             'rank_ni', 'rank_roa'],
    "流动性特征": ['rank_dolvol', 'rank_ill', 'rank_turn', 'rank_zerotrade'],
    "市场相关特征": ['market_ret', 'industry_ret', 'lag_ret_1', 'lag_ret_3'],
    "成长性特征": ['rank_rsup', 'rank_nincr', 'rank_sue', 'rank_sgr', 'rank_chpm', 'rank_lgr'],
    "风险特征": ['rank_beta', 'rank_lev'],
    "其他特征": ['rank_abr', 'rank_adm', 'rank_chtx', 'rank_pctacc', 'rank_rdm', 'rank_depr',
           'rank_pscore', 'rank_cash', 'rank_rna', 'rank_chcsho', 'rank_sp', 
           'rank_cinvest', 'rank_me_ia', 'rank_hire', 'rank_cashdebt', 'rank_me',
           'rank_grltnoa', 'log_me', 'lag_me']
}

In [17]:
class StockDataset(Dataset):
    def __init__(self, features, targets, permnos, dates):
        self.features = features
        self.targets = targets
        self.permnos = permnos  # 保留股票标识符
        self.dates = dates      # 保留日期信息
        
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx], self.permnos[idx], self.dates[idx]

In [ ]:
class EnhancedLSTMModel(nn.Module):
    def __init__(self, input_size, num_stocks, hidden_size=64, num_layers=2, output_size=1, dropout=0.2):
        super(EnhancedLSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # 股票嵌入层 - 学习不同股票的特性
        self.stock_embedding = nn.Embedding(num_stocks, 16)
        
        # LSTM层捕获时序特征
        self.lstm = nn.LSTM(
            input_size + 16,  # 输入维度增加嵌入层输出
            hidden_size, 
            num_layers, 
            batch_first=True, 
            dropout=dropout, 
            bidirectional=False
        )
        
        # 全连接层输出预测
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, 32),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(32, output_size)
        )
        
    def forward(self, x, stock_ids):
        # 获取股票嵌入
        stock_embed = self.stock_embedding(stock_ids)
        
        # 将嵌入与特征连接
        batch_size, seq_len, feat_dim = x.size()
        stock_embed_expanded = stock_embed.unsqueeze(1).expand(-1, seq_len, -1)
        x_combined = torch.cat([x, stock_embed_expanded], dim=2)
        
        # LSTM前向传播
        h0 = torch.zeros(self.num_layers, x_combined.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x_combined.size(0), self.hidden_size).to(x.device)
        
        out, _ = self.lstm(x_combined, (h0, c0))
        
        # 只取最后一个时间步的输出
        out = self.fc(out[:, -1, :])
        return out

def identify_quality_data(data):
    """识别高质量数据"""
    momentum_score = data[feature_categories["动量特征"]].mean(axis=1)
    volatility_score = 1 - data[feature_categories["波动性特征"]].mean(axis=1)
    fundamental_score = data[feature_categories["基本面特征"]].mean(axis=1)
    
    momentum_score = (momentum_score - momentum_score.min()) / (momentum_score.max() - momentum_score.min())
    volatility_score = (volatility_score - volatility_score.min()) / (volatility_score.max() - volatility_score.min())
    fundamental_score = (fundamental_score - fundamental_score.min()) / (fundamental_score.max() - fundamental_score.min())
    
    data['quality_score'] = 0.4 * momentum_score + 0.3 * volatility_score + 0.3 * fundamental_score
    quality_threshold = data['quality_score'].quantile(0.8)
    return data[data['quality_score'] >= quality_threshold]

def identify_noise_data(data):
    """识别低质量/噪声数据"""
    vol_cols = feature_categories["波动性特征"]
    vol_zscore = (data[vol_cols] - data[vol_cols].mean()) / data[vol_cols].std()
    vol_outlier = (vol_zscore > 3).any(axis=1)
    
    liquidity_cols = ['rank_dolvol', 'rank_turn']
    liquidity_score = data[liquidity_cols].mean(axis=1)
    low_liquidity = liquidity_score < liquidity_score.quantile(0.2)
    
    ret_zscore = (data['ret'] - data['ret'].mean()) / data['ret'].std()
    extreme_ret = ret_zscore.abs() > 3
    
    noise_mask = vol_outlier | low_liquidity | extreme_ret
    return data[noise_mask]

def detect_structural_change(self, stock_data, lookback=6):
        """使用贝叶斯在线变点检测(BOCPD)检测结构性变化"""
        if len(stock_data) < 12: 
            return False
        features = stock_data[['ret', 'rank_mom12m', 'rank_bm']]
        # 标准化特征
        scaler = StandardScaler()
        scaled_features = scaler.fit_transform(features)
        
        norms = np.linalg.norm(scaled_features, axis=1)
        hazard = 1/20  
        log_R = np.zeros(len(norms)  
        log_R[0] = 0  # 初始概率
        # 使用高斯分布作为似然函数
        mean = norms[0]
        var = 1e-3
        precision = 1/var
        
        for t in range(1, len(norms)):
            # 预测步
            log_R[t] = np.log(hazard) + np.max(log_R[:t])
            # 更新步
            diff = norms[t] - mean
            log_likelihood = -0.5 * (np.log(2 * np.pi) + 0.5 * np.log(precision) - 0.5 * precision * diff**2
            
            # 更新分布参数
            new_precision = precision + 1
            new_mean = (precision * mean + norms[t]) / new_precision
            
            # 更新概率
            log_R[t] += log_likelihood
            
            # 更新当前分布
            mean = new_mean
            precision = new_precision
        
        # 检测最近lookback个月内是否有变点
        recent_prob = log_R[-lookback:]
        return np.any(recent_prob > np.percentile(log_R, 95))  # 超过95%分位数视为变点

class NeuralStockForgettingModel:
    def __init__(self, 
                 input_size, 
                 stock_mapping,  # 股票ID到索引的映射
                 hidden_size=16, 
                 num_layers=1, 
                 output_size=1,
                 seq_length=8,
                 min_history=12,
                 device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        self.seq_length = seq_length
        self.min_history = min_history
        self.stock_mapping = stock_mapping
        self.model = EnhancedLSTMModel(
            input_size, 
            len(stock_mapping), 
            hidden_size, 
            num_layers, 
            output_size
        ).to(device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)
        self.criterion = nn.MSELoss()
        self.memory_bank = pd.DataFrame()
        self.predictions_df = pd.DataFrame(columns=['permno', 'date', 'pred_ret', 'memory_type', 'actual_ret'])
        self.feature_columns = [col for col in data.columns if col not in 
                               ['gvkey', 'permno', 'ret', 'date', 'ffi49', 'exchcd', 'shrcd', 'year', 'month', 'monthly_stocks']]
        self.scaler = StandardScaler()
        self.eval_metrics = {}
        self.structural_changes = {}  # 记录发生结构性变化的股票
        
    def _prepare_sequences(self, data, target_col='ret'):
        # 移除不必要的股票分组（关键优化！）
        # 直接处理全局数据，利用向量化计算替代groupby
        values = data[self.feature_columns].values
        targets = data[target_col].values
        stock_indices = data['permno'].map(self.stock_mapping).values
            
         # 标准化仅在首次调用时拟合（避免每月重复计算）
        if not hasattr(self, 'scaler_fitted'):
            self.scaler.fit(values)
            self.scaler_fitted = True
        values = self.scaler.transform(values)
            
        # 滑动窗口向量化生成序列（替代循环）
        n = len(values) - self.seq_length
        if n < 0: return [], [], [], [], []
        X = np.stack([values[i:i+self.seq_length] for i in range(n)])
        y = targets[self.seq_length-1:n+self.seq_length-1]
        dates = data['date'].values[self.seq_length-1:n+self.seq_length-1]
        permnos = data['permno'].values[self.seq_length-1:n+self.seq_length-1]
        stock_indices = stock_indices[self.seq_length-1:n+self.seq_length-1]
    
        return X, y, dates, permnos, stock_indices
    
    
    def _train_epoch(self, dataloader):
        """训练模型一个轮次"""
        self.model.train()
        total_loss = 0
        
        for features, targets, stock_ids, _ in dataloader:
            features = features.to(self.device).float()
            targets = targets.to(self.device).float().view(-1, 1)
            stock_ids = stock_ids.to(self.device).long()
            
            self.optimizer.zero_grad()
            outputs = self.model(features, stock_ids)
            loss = self.criterion(outputs, targets)
            
            loss.backward()
            self.optimizer.step()
            
            total_loss += loss.item()
            
        return total_loss / len(dataloader)
    
    def _predict_on_data(self, data, current_date, memory_type='core'):
        """对给定数据进行预测并记录结果"""
        self.model.eval()
        predictions = []
        
        with torch.no_grad():
            X, y, dates, permnos, stock_indices = self._prepare_sequences(data)
            
            if len(X) == 0:
                return
                
            features_tensor = torch.tensor(X).to(self.device).float()
            stock_ids_tensor = torch.tensor(stock_indices).to(self.device).long()
            
            outputs = self.model(features_tensor, stock_ids_tensor).cpu().numpy().flatten()
            
            for i in range(len(outputs)):
                # 检查是否为结构性变化的股票
                is_structural_change = permnos[i] in self.structural_changes
                
                predictions.append({
                    'permno': permnos[i],
                    'date': dates[i],
                    'pred_ret': outputs[i],
                    'memory_type': memory_type,
                    'actual_ret': y[i],
                    'structural_change': is_structural_change
                })
        
        if predictions:
            batch_pred = pd.DataFrame(predictions)
            self.predictions_df = pd.concat([
                self.predictions_df[~((self.predictions_df['permno'].isin(batch_pred['permno'])) & 
                                     (self.predictions_df['date'].isin(batch_pred['date'])))]
                , batch_pred
            ])
    
    def train_and_predict(self, full_data, epochs=3):
        """全数据训练并生成预测"""
        # 确保日期格式正确
        full_data['date'] = pd.to_datetime(full_data['date'])
        
        # 确定全局最小和最大日期
        global_min_date = full_data['date'].min()
        # global_max_date = full_data['date'].max()
        global_max_date = pd.to_datetime('2021-12-01')
        
        # 生成逐月的日期序列
        date_range = pd.date_range(start=global_min_date, end=global_max_date, freq='MS')
        
        # 对每个月进行处理
        for current_date in date_range:
            print(f"处理日期: {current_date}")
            
            # 获取当前月及之前的所有数据
            current_data = full_data[full_data['date'] <= current_date].copy()
            
            # 更新记忆库
            self._update_memory(current_data, current_date=current_date)
            
            # 检查记忆库中是否有足够的数据
            if len(self.memory_bank) < self.min_history:
                continue
                
            # 准备训练数据
            X_train, y_train, _, _, stock_indices = self._prepare_sequences(self.memory_bank)
            
            if len(X_train) == 0:
                continue
                
            # 创建数据加载器
            train_dataset = StockDataset(X_train, y_train, stock_indices, np.zeros(len(y_train)))
            train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
            
            # 模型训练
            for epoch in range(epochs):
                train_loss = self._train_epoch(train_loader)
                print(f"Epoch {epoch+1}/{epochs}, Loss: {train_loss:.6f}")
            
            # 早停机制（避免无效迭代）
            best_loss = np.inf
            early_stopping_count = 0
            for epoch in range(epochs):
                train_loss = self._train_epoch(train_loader)
                if train_loss > best_loss:
                    early_stopping_count += 1
                    if early_stopping_count >= 1: break  # loss上升则停止
                else:
                    best_loss = train_loss
                    early_stopping_count = 0
            
            # 对当前所有数据进行预测
            self._predict_on_data(current_data, current_date, memory_type='train')
            
            # 对当前月的数据单独预测（如果有）
            current_month_data = full_data[full_data['date'] == current_date]
            
            if not current_month_data.empty:
                self._predict_on_data(current_month_data, current_date, memory_type='test')
        
        # 计算评估指标
        self._calculate_metrics()
    
    def _update_memory(self, new_data, current_date):
        """动态更新记忆库"""
        if not self.memory_bank.empty:
            time_diff = (current_date - self.memory_bank['date']).dt.days / 30
            decay_factor = np.exp(-0.02 * time_diff)
            self.memory_bank = self.memory_bank[decay_factor >= 0.15]
            self.memory_bank = self.memory_bank[(current_date - self.memory_bank['date']) <= pd.Timedelta(days=30*24)]
        
        self.memory_bank = pd.concat([self.memory_bank, new_data], ignore_index=True).drop_duplicates()
        
        quality_data = identify_quality_data(self.memory_bank)
        self.memory_bank.loc[quality_data.index, 'memory_type'] = 'quality'
        
        noise_data = identify_noise_data(self.memory_bank)
        self.memory_bank = self.memory_bank.drop(noise_data.index, errors='ignore')
        # 检测结构性变化
        for permno in self.memory_bank['permno'].unique():
            stock_data = self.memory_bank[self.memory_bank['permno'] == permno]
            
            if detect_structural_change(stock_data):
                # 记录结构性变化
                self.structural_changes[permno] = current_date
                
                # 保留最近3个月数据，而不是全部删除
                recent_data = stock_data[stock_data['date'] > (current_date - pd.DateOffset(months=3))]
                self.memory_bank = self.memory_bank[~self.memory_bank['permno'].eq(permno) | 
                                                   self.memory_bank.index.isin(recent_data.index)]
    
    def _calculate_metrics(self):
        """计算模型评估指标"""
        valid_predictions = self.predictions_df.dropna(subset=['pred_ret', 'actual_ret'])
        
        if not valid_predictions.empty:
            y_true = valid_predictions['actual_ret']
            y_pred = valid_predictions['pred_ret']
            
            self.eval_metrics['r2'] = r2_score(y_true, y_pred)
            self.eval_metrics['mse'] = mean_squared_error(y_true, y_pred)
            self.eval_metrics['count'] = len(valid_predictions)
            
            # 单独计算结构性变化股票的指标
            if 'structural_change' in valid_predictions.columns:
                sc_predictions = valid_predictions[valid_predictions['structural_change']]
                if len(sc_predictions) > 0:
                    self.eval_metrics['r2_sc'] = r2_score(sc_predictions['actual_ret'], sc_predictions['pred_ret'])
                    self.eval_metrics['mse_sc'] = mean_squared_error(sc_predictions['actual_ret'], sc_predictions['pred_ret'])
    
    def get_predictions(self):
        """返回包含整个数据集的预测DataFrame"""
        return self.predictions_df.sort_values(['date', 'permno'])
    
    def get_metrics(self):
        """返回模型评估指标"""
        return self.eval_metrics
    
    def get_memory_stats(self):
        """获取记忆库统计信息"""
        stats = {
            'total_samples': len(self.memory_bank),
            'quality_samples': len(self.memory_bank[self.memory_bank['memory_type'] == 'quality']),
            'avg_age_months': (pd.Timestamp.now() - self.memory_bank['date']).dt.days.mean() / 30,
            'permno_coverage': self.memory_bank['permno'].nunique(),
            'structural_changes': len(self.structural_changes)
        }
        return stats

In [27]:
# 创建股票ID到索引的映射
unique_permnos = data['permno'].unique()
stock_mapping = {permno: i for i, permno in enumerate(unique_permnos)}

feature_columns = [col for col in data.columns if col not in 
                       ['gvkey', 'permno', 'ret', 'date', 'ffi49', 'exchcd', 'shrcd', 'year', 'month', 'monthly_stocks']]
# 初始化模型
input_size = len(feature_columns)
model = NeuralStockForgettingModel(
    input_size=input_size,
    stock_mapping=stock_mapping
)

# 训练并预测
model.train_and_predict(edata)

# 获取完整预测结果
all_predictions = model.get_predictions()

# 导出到Excel
all_predictions.to_excel('D:/股票项目:/memory_predictions.xlsx', index=False)

# 获取评估指标
metrics = model.get_metrics()
print(f"整体R² Score: {metrics['r2']:.4f}")
print(f"结构性变化股票R² Score: {metrics.get('r2_sc', 'N/A')}")
print(f"整体MSE: {metrics['mse']:.8f}")
print(f"结构性变化股票MSE: {metrics.get('mse_sc', 'N/A')}")

d:\Anaconda1\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


处理日期: 2016-10-01 00:00:00
Epoch 1/3, Loss: 0.046423
Epoch 2/3, Loss: 0.042641
Epoch 3/3, Loss: 0.038235
处理日期: 2016-11-01 00:00:00
Epoch 1/3, Loss: 0.033301
Epoch 2/3, Loss: 0.026525
Epoch 3/3, Loss: 0.018250
处理日期: 2016-12-01 00:00:00
Epoch 1/3, Loss: 0.020216
Epoch 2/3, Loss: 0.017925
Epoch 3/3, Loss: 0.017468
处理日期: 2017-01-01 00:00:00
Epoch 1/3, Loss: 0.016742
Epoch 2/3, Loss: 0.015013
Epoch 3/3, Loss: 0.014806
处理日期: 2017-02-01 00:00:00
Epoch 1/3, Loss: 0.014328
Epoch 2/3, Loss: 0.013822
Epoch 3/3, Loss: 0.014301
处理日期: 2017-03-01 00:00:00
Epoch 1/3, Loss: 0.013107
Epoch 2/3, Loss: 0.012406
Epoch 3/3, Loss: 0.012015
处理日期: 2017-04-01 00:00:00
Epoch 1/3, Loss: 0.010899
Epoch 2/3, Loss: 0.011199
Epoch 3/3, Loss: 0.010930
处理日期: 2017-05-01 00:00:00
Epoch 1/3, Loss: 0.009839
Epoch 2/3, Loss: 0.009549
Epoch 3/3, Loss: 0.009605
处理日期: 2017-06-01 00:00:00
Epoch 1/3, Loss: 0.009896
Epoch 2/3, Loss: 0.010391
Epoch 3/3, Loss: 0.010020
处理日期: 2017-07-01 00:00:00
Epoch 1/3, Loss: 0.009406
Epoch 2/3, L

RuntimeError: [enforce fail at alloc_cpu.cpp:115] data. DefaultCPUAllocator: not enough memory: you tried to allocate 3312164856 bytes.